In [1]:
og_data = cleaned_data_classwise
og_data 

bruh = data_encoded.loc[:, data_encoded.columns.difference(['pitchresult_category_strike_related','pitchresult_category_foul', 'pitchresult_category_in_play', 'pitchresult_category_other', 'pitchresult_category_strikeout', 'pitchresult_category_strikeout_binary', 'movement_ratio'])]

NameError: name 'cleaned_data_classwise' is not defined

In [2]:
# Filter the data for the desired columns
data = bruh

# Further filtering for FF pitchname and excluding 'field_out' eventtype
data = data[(data['pitchname'] == 'FF') & (~data['eventtype'].isin(['field_out']))]

# Filter to only include scenarios where pitchresult == 'C' (strikeout)
data_strikeout = data[data['pitchresult'] == 'C']

# Define the pitch characteristics for analysis
pitch_characteristics = ["spinrate", "horzbreak", "inducedvertbreak", "platelocside", "platelocheight", 
                         "relspeed", "average_relspeed", "relspeed_diff", "relspeed_inducedvertbreak"]

pitcher_stats = data_strikeout.groupby("pitcher").filter(lambda x: len(x) > 1)

# Group by pitcher and calculate the mean and standard deviation of the pitch characteristics
pitcher_stats = pitcher_stats.groupby("pitcher")[pitch_characteristics].agg(['mean', 'std'])

NameError: name 'bruh' is not defined

In [ ]:
pitcher_stats.columns

pitcher_stats.reset_index(inplace=True)

pitcher_stats

In [ ]:
# bruh = data_encoded.loc[:, data_encoded.columns.difference(['pitchresult_category_strike_related','pitchresult_category_foul', 'pitchresult_category_in_play', 'pitchresult_category_other', 'pitchresult_category_strikeout', 'pitchresult_category_strikeout_binary', 'movement_ratio'])]
columns = ['pitcher'] + [col for col in bruh.columns if col != 'pitcher']
bruh = bruh[columns]

# Reorder columns to make "pitcher" the first column
bruh.drop(columns=['average_relspeed'], inplace=True)
bruh

In [ ]:
bruh_1 = bruh
bruh_1

In [ ]:
# Set pitcher as the index for pitcher_stats to allow easy lookup
pitcher_stats.set_index('pitcher', inplace=True)

# Define the function to calculate the new value
def calculate_z_score_adjusted(pitch, pitcher_stats, constant=5.74):
    pitcher = pitch['pitcher']
    relspeed = pitch['relspeed']
    
    # Get pitcher's mean and std for relspeed from pitcher_stats
    try:
        relspeed_mean = pitcher_stats['relspeed']['mean'][pitcher]
        relspeed_std = pitcher_stats['relspeed']['std'][pitcher]
        
        # Calculate z-score and multiply by the constant
        z_score_adjusted = ((relspeed - relspeed_mean) / relspeed_std) * constant
        return z_score_adjusted
    except KeyError:
        return np.nan

# Apply the function to the bruh dataframe
bruh_1['relspeed_adjusted'] = bruh.apply(lambda row: calculate_z_score_adjusted(row, pitcher_stats), axis=1)

In [ ]:
bruh_1

In [ ]:
# Define the function to calculate z-score adjusted values for different characteristics
def calculate_z_score_adjusted_for_feature(pitch, pitcher_stats, feature, constant=6.17):
    pitcher = pitch['pitcher']
    value = pitch[feature]
    
    # Get pitcher's mean and std for the feature from pitcher_stats
    try:
        feature_mean = pitcher_stats[feature]['mean'][pitcher]
        feature_std = pitcher_stats[feature]['std'][pitcher]
        
        # Calculate z-score and multiply by the constant
        z_score_adjusted = ((value - feature_mean) / feature_std) * constant
        return z_score_adjusted
    except KeyError:
        return np.nan

# Apply these functions to the `bruh` dataframe
bruh_1['spinrate_adjusted'] = bruh.apply(lambda row: calculate_z_score_adjusted_for_feature(row, pitcher_stats, 'spinrate', constant=3.78366), axis=1)
bruh_1['horzbreak_adjusted'] = bruh.apply(lambda row: abs(calculate_z_score_adjusted_for_feature(row, pitcher_stats, 'horzbreak', constant =4.06374)), axis=1)
# bruh_1['inducedvertbreak_adjusted'] = bruh.apply(lambda row: calculate_z_score_adjusted_for_feature(row, pitcher_stats, 'inducedvertbreak', constant = 3.),  axis=1)
bruh_1['platelocside_adjusted'] = bruh.apply(lambda row: -calculate_z_score_adjusted_for_feature(row, pitcher_stats, 'platelocside', constant=29.64492), axis=1)
bruh_1['platelocheight_adjusted'] = bruh.apply(lambda row: calculate_z_score_adjusted_for_feature(row, pitcher_stats, 'platelocheight', constant=31.36657), axis=1)
bruh_1['relspeed_inducedvertbreak'] = bruh.apply(lambda row: calculate_z_score_adjusted_for_feature(row, pitcher_stats, 'relspeed_inducedvertbreak', constant=6.96876), axis=1)

In [ ]:
# Calculate "my_score" as the row sum of the specified columns
bruh_1['my_score'] = bruh_1[['spinrate_adjusted', 'horzbreak_adjusted', 'platelocside_adjusted', 'platelocheight_adjusted', 'relspeed_inducedvertbreak']].sum(axis=1)

# Calculate the average of "my_score" grouped by "pitcher"
average_score_by_pitcher = bruh_1.groupby('pitcher')['my_score'].mean().reset_index()

# Display the result
average_score_by_pitcher

In [ ]:
average_score_by_pitcher_sorted = average_score_by_pitcher.sort_values(by='my_score', ascending=False).reset_index(drop=True)

average_score_by_pitcher_sorted

In [ ]:
average_score_by_pitcher[average_score_by_pitcher['pitcher'] == "deGrom, Jacob"]

In [ ]:
bruh_1[bruh_1['pitcher'] == "Morton, Charlie"]

In [ ]:
plt.figure(figsize=(10, 6))
plt.hist(average_score_by_pitcher['my_score'], bins=200, edgecolor='k', alpha=0.7)
plt.xlabel('My Score')
plt.ylabel('Frequency')
plt.title('Distribution of My Score Across Pitchers')
plt.show()

In [ ]:
from pybaseball import pitching_stats

# Retrieve pitching stats for 2021 to 2023
data = pitching_stats(2021, 2023)
# Filter for desired columns
era_data = data[['Season', 'Name', 'Team', 'ERA']]

In [ ]:
era_data

In [ ]:
# Calculating the overall average ERA for each pitcher across all seasons
overall_average_era = era_data.groupby('Name')['ERA'].mean().reset_index()

overall_average_era

In [ ]:
# Adjusting the names in average_score_by_pitcher to match the format in overall_average_era

# Function to reformat "Lastname, Firstname" to "Firstname Lastname"
def reformat_name(name):
    last, first = name.split(', ')
    return f"{first} {last}"

# Applying the reformatting function
average_score_by_pitcher['Name'] = average_score_by_pitcher['pitcher'].apply(reformat_name)

# Merging the dataframes on the reformatted name column
merged_data = overall_average_era.merge(average_score_by_pitcher, on="Name")

# Plotting the ERA vs. my_score for each pitcher
plt.figure(figsize=(10, 6))
plt.scatter(merged_data['ERA'], merged_data['my_score'], color='blue', alpha=0.7)

# Adding labels and title
plt.xlabel("Average ERA")
plt.ylabel("Average My Score")
plt.title("Comparison of Average ERA and Average My Score by Pitcher")
plt.grid(True)

# Displaying the plot
plt.show()

In [ ]:
# Calculate the Pearson correlation coefficient between ERA and my_score
correlation = merged_data['ERA'].corr(merged_data['my_score'])
print("Correlation between ERA and my_score:", correlation)

In [ ]:
# Filter data to include only rows where my_score is between -20 and 20
subset_data = merged_data[(merged_data['my_score'] >= 50) & (merged_data['my_score'] <= 90)]

# Calculate the correlation for the filtered subset
subset_correlation = subset_data['ERA'].corr(subset_data['my_score'])
print("Correlation between ERA and my_score in [-20, 20]:", subset_correlation)

# Calculate the correlation for the full dataset for comparison
full_correlation = merged_data['ERA'].corr(merged_data['my_score'])
print("Correlation between ERA and my_score for all data:", full_correlation)


In [ ]:
# Plotting the ERA vs. my_score for the filtered range
plt.figure(figsize=(10, 6))
plt.scatter(subset_data['ERA'], subset_data['my_score'], color='green', alpha=0.7)

# Adding labels and title
plt.xlabel("Average ERA")
plt.ylabel("Average My Score")
plt.title("Comparison of Average ERA and My Score by Pitcher (My Score between -20 and 20)")
plt.grid(True)

# Displaying the plot
plt.show()
